## Nome: Fadhla Mohamed
## Cognome: Mutua
## Matricola: SM3201434

## Descrizione
### Crea un database che abbia 11 tabelle. Il database sarà un sistema di gestione degli scaffali:
#### Tabelle:
- 1: Lo scaffale 1 ha verdure (può contenere lattuga, pomodori, frutta, ecc.)
- 2: Lo scaffale 2 ha cibi secchi (può contenere zucchero, caffè, farina, uova, ecc.)
- 3: Lo scaffale 3 ha carne fresca (può contenere carne di manzo, agnello, ecc.)
- 4: Lo scaffale 4 ha prodotti da forno (può contenere pane, bagel, ecc.)
- 5: Lo scaffale 5 ha utensili da cucina (contiene utensili da cucina)
- 6: Lo scaffale 6 ha farmaci (contiene farmaci da banco)
- 7: Lo scaffale 7 ha cibo per animali (contiene cibo per animali)
- 8: Lo scaffale 8 ha cibi surgelati (cibo pre-porzionato come pizza + tutti i cibi degli altri scaffali che non sono cibo per animali, cibi secchi o carne fresca, con un operatore booleano che verifica se sono surgelati)
- 9: Lo scaffale 9 ha cibi refrigerati (può essere latticini, yogurt, cibi che necessitano refrigerazione)
- 10: Lo scaffale 10 ha bevande
- 11: Lo scaffale 11 ha snack
#### Data la struttura del database, desidero consentire alcuni cose:
- 1: Alcune bevande possono essere nello scaffale refrigerato (ad esempio cola fredda e cola a temperatura ambiente)
- 2: Alcune verdure possono essere nello scaffale surgelato (ad esempio patatine fritte precotte)
- 3: Alcuni cibi per animali devono essere nello scaffale dei farmaci (ad esempio cibo dietetico per animali)
- 4: Alcune carni fresche possono essere nello scaffale refrigerato (ad esempio salmone, carne rossa)
- 5: Alcuni snack possono essere negli scaffali di cibi secchi (ad esempio pistacchi, noci)
- 6: Alcuni prodotti da forno possono essere nello scaffale refrigerato (ad esempio torte)
- 7: Tutti gli articoli dello scaffale 5 (cibo in scatola) sono una miscela di verdure e carne, con un operatore booleano che verifica se sono in scatola
#### Ogni articolo ha un numero identificativo unico e più articoli possono provenire dalla stessa azienda ed essi appartengono ad un unico banco.


## Operazioni

## Trigger e Funzioni e Procedure

### 1. Trigger `trg_check_item_aisle_count`
Lo trigger `trg_check_item_aisle_count` è usato per verificare che un item possa appartenere solo a un unico banco. Se un item è già presente in un altro banco, l'inserimento viene impedito e viene sollevato un errore.
```sql{
DELIMITER $$
CREATE TRIGGER trg_check_item_aisle_count
BEFORE INSERT ON Contains
FOR EACH ROW
BEGIN
    DECLARE aisle_count INT;

    -- Check if the ItemID already exists in another aisle (excluding the same aisle)
    SELECT COUNT(*) INTO aisle_count
    FROM Contains
    WHERE ItemID = NEW.ItemID AND AisleID != NEW.AisleID;

    -- If the item already exists in another aisle, raise an error
    IF aisle_count > 0 THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'An item can only belong to one aisle.';
    END IF;
END$$
DELIMITER ;
```

### 2. Trigger `EnsureSingleManufacturer`
Lo trigger `EnsureSingleManufacturer` è utilizzato per assicurarsi che ogni item sia fabbricato da una sola azienda. Se un item esiste già nella tabella `Manufactured_By`, l'inserimento viene annullato e viene sollevato un errore.
```sql{
DELIMITER $$
CREATE TRIGGER EnsureSingleManufacturer
BEFORE INSERT ON Manufactured_By
FOR EACH ROW
BEGIN
    -- Check if the ItemID already exists in the Manufactured_By table
    IF EXISTS (
        SELECT 1
        FROM Manufactured_By
        WHERE ItemID = NEW.ItemID
    ) THEN
        SIGNAL SQLSTATE '45000'
        SET MESSAGE_TEXT = 'Each Item can only be manufactured by one Company.';
    END IF;
END$$
DELIMITER ;
}
```

### 3. Funzione `fn_validate_aisle_compliance`
La funzione `fn_validate_aisle_compliance` valida la conformità di un item rispetto al banco in cui è stato inserito, controllando che il tipo di conservazione e la categoria dell'item corrispondano alle regole dei banchi. Se non è conforme, restituisce un messaggio di errore.
```sql{
DELIMITER $$
CREATE FUNCTION fn_validate_aisle_compliance(item_id INT, aisle_id INT)
RETURNS VARCHAR(255)
DETERMINISTIC
BEGIN
    DECLARE item_storage_type VARCHAR(50);
    DECLARE item_category VARCHAR(50);
    DECLARE error_message VARCHAR(255);

    -- Fetch storage type and category
    SELECT StorageType, Category INTO item_storage_type, item_category
    FROM Item
    WHERE ItemID = item_id;

    -- Initialize error message
    SET error_message = NULL;

    -- Validate based on storage type and category
    CASE
        -- Frozen storage type validation
        WHEN item_storage_type = 'Frozen' AND
             (aisle_id <> (SELECT AisleID FROM Aisle WHERE Name = 'Frozen') OR
              item_category NOT IN ('Frozen', 'Fresh Meat')) THEN
            SET error_message = CONCAT(item_category, ' items with Frozen storage must be placed in the Frozen aisle.');

        -- Refrigerated storage type validation
        WHEN item_storage_type = 'Refrigerated' AND
             item_category IN ('Baked', 'Fresh Meat', 'Beverages') AND
             aisle_id <> (SELECT AisleID FROM Aisle WHERE Name = 'Refrigerated') THEN
            SET error_message = CONCAT(item_category, ' items with Refrigerated storage must be placed in the Refrigerated aisle.');

        -- Ambient storage type validation
        WHEN item_storage_type = 'Ambient' AND
             item_category IN ('Refrigerated', 'Frozen') THEN
            SET error_message = CONCAT(item_category, ' items cannot have Ambient storage type.');

        -- Vegetables validation
        WHEN item_category = 'Vegetables' AND
             aisle_id IN (SELECT AisleID FROM Aisle WHERE Name IN ('Frozen', 'Refrigerated')) THEN
            SET error_message = 'Vegetables cannot be placed in Frozen or Refrigerated aisles.';

        -- Kitchenware validation
        WHEN item_category = 'Kitchenware' AND
             aisle_id <> (SELECT AisleID FROM Aisle WHERE Name = 'Kitchenware') THEN
            SET error_message = 'Kitchenware items must be placed in the Kitchenware aisle.';

        -- Pet Food validation
        WHEN item_category = 'Pet Food' AND
             aisle_id NOT IN (SELECT AisleID FROM Aisle WHERE Name IN ('Pet Food', 'Pharmacy')) THEN
            SET error_message = 'Pet Food must be placed in the Pet Food or Pharmacy aisle.';

        -- Dry Food, Baked, Beverages, and Snacks validation
        WHEN item_category IN ('Baked', 'Beverages') AND
             item_storage_type = 'Frozen' THEN
            SET error_message = CONCAT(item_category, ' items cannot be stored in Frozen aisles.');

        WHEN item_category in ('Dry Food', 'Snacks') AND item_storage_type
                in ('Frozen', 'Refrigerated') THEN
            SET error_message = CONCAT(item_category, ' items cannot be stored in Frozen/Refrigerated aisles.');

        -- Catch-all for unexpected cases
        ELSE
            SET error_message = NULL;
    END CASE;

    RETURN error_message;
END$$
DELIMITER ;
}
```

### 4. Funzione `fn_suggest_correct_aisle`
La funzione `fn_suggest_correct_aisle` suggerisce l'area corretta in cui inserire un item, basandosi sul tipo di conservazione e sulla categoria dell'item. Restituisce l'ID del banco che è più adatto per l'item.
```sql{
DELIMITER $$
CREATE FUNCTION fn_suggest_correct_aisle(item_id INT)
RETURNS INT
DETERMINISTIC
BEGIN
    DECLARE item_storage_type VARCHAR(50);
    DECLARE item_category VARCHAR(50);
    DECLARE suggested_aisle INT;

    -- Fetch storage type and category
    SELECT StorageType, Category INTO item_storage_type, item_category
    FROM Item
    WHERE ItemID = item_id;

    -- Determine the correct aisle
    CASE
        WHEN item_storage_type = 'Frozen' THEN
            SELECT AisleID INTO suggested_aisle FROM Aisle WHERE Name = 'Frozen';
        WHEN item_storage_type = 'Refrigerated' THEN
            SELECT AisleID INTO suggested_aisle FROM Aisle WHERE Name = 'Refrigerated';
        WHEN item_category = 'Vegetables' THEN
            SELECT AisleID INTO suggested_aisle FROM Aisle WHERE Name = 'Vegetables';
        WHEN item_category = 'Kitchenware' THEN
            SELECT AisleID INTO suggested_aisle FROM Aisle WHERE Name = 'Kitchenware';
        WHEN item_category = 'Pet Food' THEN
            SELECT AisleID INTO suggested_aisle FROM Aisle WHERE Name IN ('Pet Food', 'Pharmacy') LIMIT 1;
        ELSE
            SELECT AisleID INTO suggested_aisle FROM Aisle WHERE Name = 'Ambient';
    END CASE;

    RETURN suggested_aisle;
END$$
DELIMITER ;
```

### 5. Procedura `pr_insert_item_log`
La procedura `pr_insert_item_log` inserisce un log di errore nella tabella `ItemLogErrors` ogni volta che un item viene inserito in un banco sbagliato. La procedura include anche un messaggio di errore e suggerimenti per il banco corretto.
```sql{
DELIMITER $$
CREATE PROCEDURE pr_insert_item_log(item_id INT, aisle_id INT, error_message VARCHAR(255))
BEGIN
    -- Insert the log entry into the ItemLog table
    INSERT INTO ItemLogErrors (ItemID, AisleID, LogTime, ErrorID)
    VALUES (item_id, aisle_id, NOW(),error_message);
END$$
DELIMITER ;
```

### 5.5 Funzione `fn_insert_into_error_message`
Funzione ausiliario usato per introdurre nouvi tipi di errori e chiamare quelli gia registrati 
```sql{
DELIMITER $$
CREATE FUNCTION fn_insert_into_error_message(error_message VARCHAR(255))
RETURNS INT
DETERMINISTIC
BEGIN
    DECLARE error_id INT;
    -- Insert the log entry into the ItemLog table
   IF NOT EXISTS (
            SELECT 1
            FROM ErrorMessages
            WHERE ErrorMessage = error_message
        ) THEN
            -- If not, insert the error message into the ErrorMessages table
            INSERT INTO ErrorMessages (ErrorMessage)
            VALUES (error_message);
        END IF;
    select ErrorID into error_id
        from ErrorMessages
        where ErrorMessage = error_message;
    RETURN error_id;
END$$
DELIMITER ;
```

### 6. Trigger `trg_log_item_wrong_aisle`
Lo trigger `trg_log_item_wrong_aisle` è utilizzato per registrare errori quando un item viene inserito in un banco sbagliato. Se l'item non rispetta le regole di conformità, viene generato un messaggio di errore e viene suggerito un banco corretto.
```sql{
DELIMITER $$
CREATE TRIGGER trg_log_item_wrong_aisle
AFTER INSERT ON Contains
FOR EACH ROW
BEGIN
    DECLARE error_message VARCHAR(255);
    DECLARE error_id INT;

    -- Call the validation function to check compliance
    set error_message = fn_validate_aisle_compliance(NEW.ItemID, NEW.AisleID);

    -- If non-compliant, suggest correct aisle, log the error, and reject the insertion
    IF error_message IS NOT NULL THEN
        -- Call the suggestion function to get the correct aisle

        SET error_id = fn_insert_into_error_message(error_message);
        -- Call the insertion procedure to log the incorrect insertion
        CALL pr_insert_item_log(NEW.ItemID,NEW.AisleID, error_id);

    END IF;
END$$
DELIMITER ;
```

## Inserimenti

### 1. Inserimento nella tabella `Aisle`
L'inserimento nella tabella `Aisle` definisce i vari banchi, ognuno identificato da un ID e un nome. I banchi sono categorizzati per tipo.
```sql{
INSERT INTO Aisle (AisleID, Name)
VALUES
    (1, 'Vegetables'),
    (2, 'Dry Food'),
    (3, 'Fresh Meat'),
    (4, 'Baked'),
    (5, 'Kitchenware'),
    (6, 'Pharmacy'),
    (7, 'Pet Food'),
    (8, 'Frozen'),
    (9, 'Refrigerated'),
    (10, 'Beverages'),
    (11, 'Snacks');
```

### 2. Inserimento nella tabella `Company`
L'inserimento nella tabella `Company` aggiunge diverse aziende, ognuna con un ID, un nome, un'email e una localizzazione.
```sql{
INSERT INTO Company (CompanyID, Name,Email, Location)
VALUES
    (1, 'Fresh Farms', 'company1@blabla.bla', 'France'),
    (2, 'Dry Goods Co.', 'company2@blabla.bla', 'Germany'),
    (3, 'Meat Masters', 'company3@blabla.bla', 'Italy'),
    (4, 'Bakers Delight', 'company4@blabla.bla', 'Spain'),
    (5, 'KitchenPro', 'company5@blabla.bla', 'Netherlands');
```

### 3. Inserimento nella tabella `Item`
L'inserimento nella tabella `Item` aggiunge vari articoli con dettagli come nome, categoria, tipo di conservazione e data di scadenza.
```sql{
INSERT INTO Item (Name, Category, StorageType, ExpirationDate)
VALUES
    -- Aisle 1: Vegetables
    ('Lettuce', 'Vegetables', 'Ambient', '2025-01-20 12:00:00'),
    ('Tomatoes', 'Vegetables', 'Ambient', '2025-01-22 12:00:00'),
    ('Potatoes', 'Vegetables', 'Ambient', '2025-02-01 12:00:00'),

    -- Aisle 2: Dry Food
    ('Sugar', 'Dry Food', 'Ambient', '2025-12-31 12:00:00'),
    ('Coffee', 'Dry Food', 'Ambient', '2026-01-15 12:00:00'),
    ('Flour', 'Dry Food', 'Ambient', '2025-11-30 12:00:00'),

    -- Aisle 3: Fresh Meat
    ('Beef', 'Fresh Meat', 'Ambient', '2025-01-19 12:00:00'),
    ('Lamb', 'Fresh Meat', 'Ambient', '2025-01-20 12:00:00'),
    ('Salmon', 'Fresh Meat', 'Ambient', '2025-01-18 12:00:00'),

    -- Aisle 4: Baked
    ('Bread', 'Baked', 'Ambient', '2025-01-25 12:00:00'),
    ('Bagels', 'Baked', 'Ambient', '2025-01-24 12:00:00'),
    ('Cakes', 'Baked', 'Refrigerated', '2025-01-23 12:00:00'),

    -- Aisle 5: Kitchenware
    ('Cooking Pot', 'Kitchenware', 'Ambient', '2100-01-01 12:00:00'),
    ('Knife Set', 'Kitchenware', 'Ambient', '2100-01-01 12:00:00'),
    ('Blender', 'Kitchenware', 'Ambient', '2100-01-01 12:00:00'),

    -- Aisle 6: Pharmacy
    ('Aspirin', 'Pharmacy', 'Ambient', '2025-06-30 12:00:00'),
    ('Bandages', 'Pharmacy', 'Ambient', '2025-07-01 12:00:00'),
    ('Diet Pet Food', 'Pet Food', 'Ambient', '2025-05-30 12:00:00'),

    -- Aisle 7: Pet Food
    ('Dog Food', 'Pet Food', 'Ambient', '2025-04-30 12:00:00'),
    ('Cat Food', 'Pet Food', 'Ambient', '2025-04-30 12:00:00'),
    ('Fish Food', 'Pet Food', 'Ambient', '2025-04-30 12:00:00'),

    -- Aisle 8: Frozen
    ('Frozen Pizza', 'Frozen', 'Frozen', '2025-03-31 12:00:00'),
    ('Frozen Berries', 'Frozen', 'Frozen', '2025-03-31 12:00:00'),
    ('Frozen Beef', 'Fresh Meat', 'Frozen', '2025-03-31 12:00:00'),

    -- Aisle 9: Refrigerated
    ('Milk', 'Dairy', 'Refrigerated', '2025-01-21 12:00:00'),
    ('Yogurt', 'Dairy', 'Refrigerated', '2025-01-22 12:00:00'),
    ('Cheese', 'Dairy', 'Refrigerated', '2025-01-23 12:00:00'),

    -- Aisle 10: Beverages
    ('Cola', 'Beverages', 'Ambient', '2025-08-30 12:00:00'),
    ('Cold Cola', 'Beverages', 'Ambient', '2025-08-30 12:00:00'),
    ('Orange Juice', 'Beverages', 'Ambient', '2025-08-30 12:00:00'),

    -- Aisle 11: Snacks
    ('Chips', 'Snacks', 'Ambient', '2025-09-15 12:00:00'),
    ('Nuts', 'Snacks', 'Ambient', '2025-10-01 12:00:00'),
    ('Chocolate', 'Snacks', 'Ambient', '2025-09-20 12:00:00');
```

### 4. Inserimento nella tabella `Contains`
L'inserimento nella tabella `Contains` associa gli articoli ai banchi, specificando in quale banco ogni articolo è presente.
```sql{
INSERT INTO Contains (AisleID, ItemID)
VALUES
    -- Aisle 1: Vegetables
    (1, 1), (1, 2), (1, 3),
    -- Aisle 2: Dry Food
    (2, 4), (2, 5), (2, 6),
    -- Aisle 3: Fresh Meat
    (3, 7), (3, 8), (3, 9),
    -- Aisle 4: Baked
    (4, 10), (4, 11), (4, 12),
    -- Aisle 5: Kitchenware
    (5, 13), (5, 14), (5, 15),
    -- Aisle 6: Pharmacy
    (6, 16), (6, 17), (6, 18),
    -- Aisle 7: Pet Food
    (7, 19), (7, 20), (7, 21),
    -- Aisle 8: Frozen (only frozen items should be here)
    (8, 22), (8, 23), (8, 24),
    -- Aisle 9: Refrigerated (only refrigerated items should be here)
    (9, 25), (9, 26), (9, 27),
    -- Aisle 10: Beverages
    (10, 28), (10, 29), (10, 30),
    -- Aisle 11: Snacks
    (11, 31), (11, 32), (11, 33);
```

### 5. Inserimento nella tabella `Manufactured_By`
L'inserimento nella tabella `Manufactured_By` stabilisce quale azienda ha prodotto ciascun item, legando ogni item a una compagnia specifica.
```sql{
INSERT INTO Manufactured_By (ItemID, CompanyID)
VALUES
    (1, 1), (2, 1), (3, 1),
    (4, 2), (5, 2), (6, 2),
    (7, 3), (8, 3), (9, 3),
    (10, 4), (11, 4), (12, 4),
    (13, 5), (14, 5), (15, 5),
    (16, 1), (17, 1), (18, 1),
    (19, 2), (20, 2), (21, 2),
    (22, 3), (23, 3), (24, 3),
    (25, 4), (26, 4), (27, 4),
    (28, 5), (29, 5), (30, 5),
    (31, 1), (32, 1), (33, 1);
```

## Test del Trigger
Verifica se il trigger aggiunge un Item in ``ItemLogErrors`` se l'Item e nel banco sbagliato
```sql{
INSERT INTO Item
value (34,'Canned beef','Dry food','Refrigerated','2025-02-01');

insert into contains
    value (2,34);

select * from ItemLogErrors;
```

## Join

### 1. Inner Join
La query con `INNER JOIN` seleziona gli item che sono presenti in uno o più banchi. Vengono mostrati solo gli articoli che hanno una corrispondenza nelle tabelle `Item`, `Contains` e `Aisle`.
```sql{
SELECT i.ItemID, i.Name AS ItemName, a.Name AS AisleName
FROM Item i
INNER JOIN Contains c ON i.ItemID = c.ItemID
INNER JOIN Aisle a ON c.AisleID = a.AisleID;
```

### 2. Left Join
La query con `LEFT JOIN` mostra tutti gli articoli, anche quelli che non sono associati a nessun banco. I banchi che non hanno articoli associati sono mostrati come `NULL`.
```sql{
SELECT i.ItemID, i.Name AS ItemName, a.Name AS AisleName
FROM Item i
LEFT JOIN Contains c ON i.ItemID = c.ItemID
LEFT JOIN Aisle a ON c.AisleID = a.AisleID;
```

### 3. Right Join
La query con `RIGHT JOIN` mostra tutti i banchi, anche quelli che non contengono articoli. Gli articoli non associati sono mostrati come `NULL`.
```sql{
SELECT a.Name AS AisleName, i.ItemID, i.Name AS ItemName
FROM Aisle a
RIGHT JOIN Contains c ON a.AisleID = c.AisleID
RIGHT JOIN Item i ON c.ItemID = i.ItemID;
```

### 4. Full/Outer Join
La query con `FULL OUTER JOIN` restituisce tutti gli articoli e tutti i banchi, unendo i dati provenienti da entrambe le tabelle, anche quando non c'è corrispondenza tra di loro.
```sql{
SELECT i.ItemID, i.Name AS ItemName, a.Name AS AisleName
FROM Item i
LEFT JOIN Contains c ON i.ItemID = c.ItemID
LEFT JOIN Aisle a ON c.AisleID = a.AisleID
UNION
SELECT i.ItemID, i.Name AS ItemName, a.Name AS AisleName
FROM Aisle a
RIGHT JOIN Contains c ON a.AisleID = c.AisleID
RIGHT JOIN Item i ON c.ItemID = i.ItemID;
```

### 5. Items in Ogni Banco
La query per "Items in Each Aisle" mostra il numero di articoli presenti in ogni banco, utilizzando un `JOIN` tra le tabelle `Aisle` e `Contains`.
```sql{
SELECT a.Name AS Aisle, COUNT(c.ItemID) AS NumberOfItems
FROM Aisle a
JOIN Contains c ON a.AisleID = c.AisleID
GROUP BY a.AisleID;
```

### 6. Articoli per Tipo di Stoccaggio
La query "Items by Storage Type" mostra il numero di articoli per ciascun tipo di stoccaggio (ad esempio, congelato, refrigerato, o ambiente), raggruppando per `StorageType` nella tabella `Item`.
```sql{
SELECT i.StorageType, COUNT(i.ItemID) AS NumberOfItems
FROM Item i
GROUP BY i.StorageType;
```

### 7. Articoli in Ogni Banco per Tipo di Stoccaggio
La query "Items in Each Aisle for Each Storage Type" mostra il numero di articoli per ciascun tipo di stoccaggio in ogni banco. Utilizza un `JOIN` tra le tabelle `Aisle`, `Contains`, e `Item` per ottenere i risultati.
```sql{
SELECT a.Name AS Aisle, i.StorageType, COUNT(c.ItemID) AS NumberOfItems
FROM Aisle a
JOIN Contains c ON a.AisleID = c.AisleID
JOIN Item i ON c.ItemID = i.ItemID
GROUP BY a.AisleID, i.StorageType;
```

### 8. Articoli in Ogni Banco per Categoria
La query "Items in Each Aisle by Category" conta gli articoli per categoria in ogni banco, raggruppando per `Category` nella tabella `Item` e utilizzando un `JOIN` con la tabella `Contains` per identificare la posizione.
```sql{
SELECT a.Name AS Aisle, i.Category, COUNT(c.ItemID) AS NumberOfItems
FROM Aisle a
JOIN Contains c ON a.AisleID = c.AisleID
JOIN Item i ON c.ItemID = i.ItemID
GROUP BY a.AisleID, i.Category;
```

### 9. Elenco degli Articoli in Ogni Banco
La query "List of Items in Each Aisle" restituisce un elenco di articoli presenti in ciascun banco, con l'ID dell'articolo, il nome dell'articolo e il nome del banco in cui è collocato, utilizzando i `JOIN` tra le tabelle `Item`, `Contains`, e `Aisle`.
```sql{
SELECT i.ItemID, i.Name AS ItemName, a.Name AS AisleName
FROM Item i
JOIN Contains c ON i.ItemID = c.ItemID
JOIN Aisle a ON c.AisleID = a.AisleID;
```

### 10. Articoli Non nel Banco Suggerito
La query "Items That Are Not in the Suggested Aisle" mostra gli articoli che sono collocati nel banco sbagliato. Utilizza una logica condizionale `CASE` per determinare se l'articolo è nel banco giusto in base al suo `StorageType`.
```sql{
SELECT i.ItemID, i.Name AS ItemName, a.Name AS AisleName,
       CASE
           WHEN i.StorageType = 'Frozen' AND a.Name <> 'Frozen' THEN 'Wrong Aisle'
           WHEN i.StorageType = 'Refrigerated' AND a.Name <> 'Refrigerated' THEN 'Wrong Aisle'
           WHEN i.StorageType = 'Ambient' AND a.Name IN ('Frozen', 'Refrigerated') THEN 'Wrong Aisle'
           ELSE 'Correct Aisle'
       END AS AisleStatus
FROM Item i
JOIN Contains c ON i.ItemID = c.ItemID
JOIN Aisle a ON c.AisleID = a.AisleID;
```

### 11. Elenco Articoli per Tipo di Stoccaggio e Banco
La query "List Items by Storage Type and Aisle" mostra gli articoli per tipo di stoccaggio e per banco, ordinando i risultati prima per `StorageType` e poi per nome del banco.
```sql{
SELECT i.StorageType, a.Name AS AisleName, i.Name AS ItemName
FROM Item i
JOIN Contains c ON i.ItemID = c.ItemID
JOIN Aisle a ON c.AisleID = a.AisleID
ORDER BY i.StorageType, a.Name;
```

## Stored Procedures

### Procedura per Verificare la Conformità dell'Articolo
La procedura `CheckItemCompliance` verifica se un articolo è collocato nel banco suggerito in base alla sua categoria e tipo di stoccaggio. Se l'articolo non è nel banco giusto, restituisce un messaggio di errore.
```sql{
DELIMITER $$
CREATE PROCEDURE CheckItemCompliance(IN item_id INT, IN aisle_id INT)
BEGIN
    DECLARE item_storage_type VARCHAR(50);
    DECLARE item_category VARCHAR(50);
    DECLARE error_message VARCHAR(255);
    DECLARE suggested_aisle INT;

    -- Fetch storage type and category for the item
    SELECT StorageType, Category INTO item_storage_type, item_category
    FROM Item
    WHERE ItemID = item_id;

    -- Get the suggested aisle from the function
    SET suggested_aisle = fn_suggest_correct_aisle(item_id);

    -- Initialize error message
    SET error_message = NULL;

    -- Check if the aisle provided matches the suggested aisle
    IF aisle_id <> suggested_aisle THEN
        SET error_message = CONCAT('Item with category "', item_category, '" should be placed in aisle ', suggested_aisle, '.');
    END IF;

    -- Output the error message or compliance status
    IF error_message IS NOT NULL THEN
        SELECT error_message AS ComplianceError;
    ELSE
        SELECT 'Item is compliant' AS ComplianceStatus;
    END IF;

    -- The trigger trg_log_item_compliance will automatically handle logging of compliance errors on insert
    -- if a violation is detected and will reject the insert with an error message.
END$$
DELIMITER ;
```

### Procedura per Verificare la Collocazione degli Articoli
La procedura `sp_check_item_placement` verifica la collocazione di tutti gli articoli nel sistema. Utilizza un cursore per esaminare ciascun articolo e la sua collocazione, suggerendo se l'articolo è nel banco giusto. Se ci sono errori, li registra nel log degli errori.
```sql{
DELIMITER //
CREATE PROCEDURE sp_check_item_placement()
BEGIN
    DECLARE item_id INT(100);
    DECLARE aisle_id INT(100);
    DECLARE item_storage_type VARCHAR(50);
    DECLARE item_category VARCHAR(50);
    DECLARE aisle_name VARCHAR(50);
    DECLARE suggested_aisle INT;
    DECLARE error_message VARCHAR(255);
    DECLARE error_id INT;

    -- Cursor to iterate over items and their aisle placement
    DECLARE done INT DEFAULT 0;
    DECLARE cur CURSOR FOR
        SELECT I.ItemID, I.StorageType, I.Category, A.Name AS AisleName, C.AisleID
        FROM Item I
        JOIN Contains C ON I.ItemID = C.ItemID
        JOIN Aisle A ON C.AisleID = A.AisleID;

    -- Handler for cursor end
    DECLARE CONTINUE HANDLER FOR NOT FOUND SET done = 1;

    OPEN cur;

    -- Iterate through each row from the cursor
    read_loop: LOOP
        FETCH cur INTO item_id, item_storage_type, item_category, aisle_name, aisle_id;

        IF done THEN
            LEAVE read_loop;
        END IF;

        -- Suggest the correct aisle
        SET suggested_aisle = fn_suggest_correct_aisle(@item_id);

        -- Check if the item is placed in the suggested aisle
        IF (@aisle_id <> suggested_aisle) THEN
            SET error_message = CONCAT(item_category, ' item is not in the suggested aisle: ', suggested_aisle);
        END IF;

        -- If there is an error, log it
        IF error_message IS NOT NULL THEN
            SET error_id = fn_insert_into_error_message(error_message);
            INSERT INTO ItemLogErrors (ItemID, AisleID, LogTime, ErrorID)
            VALUES (@item_id, @aisle_id, NOW(), error_id);
        END IF;
    END LOOP;

    CLOSE cur;
END //
DELIMITER ;
```

## Eventi

### Eventi per la Gestione della Scadenza degli Articoli
L'evento `ExpirationCheckEvent` viene eseguito ogni giorno per verificare gli articoli scaduti. Se trova articoli scaduti, invia un avviso all'azienda produttrice e registra l'errore nel log `ItemLog`.
```sql{
DELIMITER $$
CREATE EVENT ExpirationCheckEvent
ON SCHEDULE EVERY 1 DAY -- Run the event every day
STARTS CURRENT_TIMESTAMP -- Start the event immediately
DO
BEGIN
    DECLARE done INT DEFAULT 0;
    DECLARE item_id INT;
    DECLARE item_name VARCHAR(100);
    DECLARE item_expiration DATETIME;
    DECLARE company_id INT;
    DECLARE company_name VARCHAR(100);
    DECLARE company_email VARCHAR(255);
    DECLARE error_message VARCHAR(255);
    DECLARE error_id INT;

    -- Declare a cursor to fetch expired items
    DECLARE expired_items CURSOR FOR
        SELECT ItemID, Name, ExpirationDate
        FROM Item
        WHERE ExpirationDate <= NOW() AND ExpirationDate > '1000-01-01'; -- Avoid items with invalid expiration date

    -- Declare a handler to exit the cursor loop
    DECLARE CONTINUE HANDLER FOR NOT FOUND SET done = 1;

    OPEN expired_items;

    -- Loop through the expired items
    read_loop: LOOP
        FETCH expired_items INTO item_id, item_name, item_expiration;
        IF done THEN
            LEAVE read_loop;
        END IF;

        -- Get the CompanyID, Name, and Email from the Manufactured_By and Company tables
        SELECT mb.CompanyID, c.Name, c.Email INTO company_id, company_name, company_email
        FROM Manufactured_By mb
        JOIN Company c ON mb.CompanyID = c.CompanyID
        WHERE mb.ItemID = item_id
        LIMIT 1;

        -- Insert log entry if company ID is found
        IF company_id IS NOT NULL THEN
            SET error_message = CONCAT('The item "', item_name, '" has expired. Company: ', company_name, ', Email: ', company_email);
            SET error_id = fn_insert_into_error_message(error_message);
            INSERT INTO ItemLog (ItemID, CompanyID, LogTime, ErrorID)
            VALUES (item_id, company_id,NOW(), error_id);
        END IF;
    END LOOP;

    CLOSE expired_items;
END$$
DELIMITER ;
```

### Esecuzione delle Procedura e degli Eventi
La procedura `CheckItemCompliance` e `sp_check_item_placement` vengono invocate per verificare la conformità degli articoli e la loro collocazione. Inoltre, l'evento `ExpirationCheckEvent` viene eseguito quotidianamente per controllare gli articoli scaduti.
```sql{
INSERT INTO Item (Name, Category, StorageType, ExpirationDate)
VALUE
    ('Salmon', 'Fresh Meat', 'Ambient', '2025-01-18 12:00:00');
SELECT
    il.*,
    em.ErrorMessage AS ErrorMessages
FROM
    ItemLog il
LEFT JOIN
    ErrorMessages em
ON
    il.ErrorID = em.ErrorID;
```



### Schema Concetuale
| **Entitá**        | **Attributi**                                                                                      |
|-------------------|----------------------------------------------------------------------------------------------------|
| **Aisle**         | AisleID (PK), Name                                                                                 |
| **Item**          | ItemID (PK), Name, Category, StorageType, ExpirationDate                                           |
| **Company**       | CompanyID (PK), Name, Email, Location                                                              |
| **ItemLog**       | LogID (PK), ItemID (FK), CompanyID (FK), ErrorMessage, LogTime                                      |
| **ItemLogErrors** | LogID (PK), ItemID (FK), AisleID (FK), ErrorMessage, SuggestedAisle, LogTime                       |

---

| **Relazione**          | **Tipo di Relazione**                                                    | **Attributi**                                                       |
|---------------------------|---------------------------------------------------------------------------|---------------------------------------------------------------------|
| **Contains**              | Aisle (0;1) - (1;N) Item                                                      | None                                                                |
| **Manufactured_By**       | Item (1;N) - (0;1) Company                                                    | None                                                                |
| **ItemLog**               | Item (0;N) - (0;1) Company                       | LogTime, ErrorMessage                                               |
| **ItemLogErrors**         | Item (0;N) - (0;1) Aisle              | ErrorMessage, SuggestedAisle, LogTime                               |


## Analisi della Ridondanza per lo Schema del Database

### 1. Tabella Aisle (Corsia):
   - **Attributi:** AisleID (PK), Name (Nome)
   - **Analisi:** Nessuna ridondanza. Ogni corsia ha un AisleID unico e un Nome. Questi attributi non vengono ripetuti altrove.

### 2. Tabella Item (Oggetto):
   - **Attributi:** ItemID (PK), Name (Nome), Category (Categoria), StorageType (Tipo di Conservazione), ExpirationDate (Data di Scadenza)
   - **Analisi:** Nessuna ridondanza nella tabella Item. Gli attributi sono distinti per ciascun oggetto e sono necessari per descrivere ogni articolo.

### 3. Tabella Company (Azienda):
   - **Attributi:** CompanyID (PK), Name (Nome), Email, Location (Posizione)
   - **Analisi:** Nessuna ridondanza nella tabella Company. I dettagli dell'azienda sono univocamente memorizzati.

### 4. Tabella ItemLog (Registro Oggetto):
   - **Attributi:** LogID (PK), ItemID (FK), CompanyID (FK), ErrorMessage (Messaggio di Errore), LogTime (Ora del Log)
   - **Analisi:**
     - **Possibile Ridondanza:** Se lo stesso oggetto della stessa azienda genera più log con lo stesso **ErrorMessage**, i campi **ItemID**, **CompanyID** e **ErrorMessage** potrebbero contenere informazioni ripetute. E quindi considero la Normalizzazione creando **ErrorMessage** in una tabella separata e lo collego tramite una chiave esterna per ridurre la ridondanza nella tabella **ItemLog**.

### 5. Tabella ItemLogErrors (Errori nel Registro Oggetti):
   - **Attributi:** LogID (PK), ItemID (FK), AisleID (FK), ErrorMessage (Messaggio di Errore), SuggestedAisle (Corsia Suggerita), LogTime (Ora del Log)
   - **Analisi:**
     - **Possibile Ridondanza:** Simile a **ItemLog**, i campi **ItemID**, **AisleID** e **ErrorMessage** potrebbero essere ripetuti per lo stesso oggetto in più record. Anche qui considero la Normalizzazione creando **ErrorMessage** in una tabella separata per evitare la memorizzazione ridondante.


### Schema Logica
| **Table**         | **Attributes**                                                                                                  |
|-------------------|------------------------------------------------------------------------------------------------------------------|
| **Aisle**         | AisleID (PK) INT, Name VARCHAR(100) NOT NULL                                                                    |
| **Item**          | ItemID (PK) INT AUTO_INCREMENT, Name VARCHAR(100) NOT NULL, Category VARCHAR(50) NOT NULL, StorageType VARCHAR(50), ExpirationDate DATETIME NOT NULL |
| **Company**       | CompanyID (PK) INT AUTO_INCREMENT, Name VARCHAR(100) NOT NULL, Email VARCHAR(255) NOT NULL, Location VARCHAR(100)|
| **Contains**      | AisleID (FK) INT, ItemID (FK) INT, PRIMARY KEY (AisleID, ItemID), FOREIGN KEY (AisleID) REFERENCES Aisle(AisleID), FOREIGN KEY (ItemID) REFERENCES Item(ItemID) |
| **Manufactured_By** | ItemID (FK) INT, CompanyID (FK) INT, PRIMARY KEY (ItemID, CompanyID), FOREIGN KEY (ItemID) REFERENCES Item(ItemID), FOREIGN KEY (CompanyID) REFERENCES Company(CompanyID) |
| **ItemLog**       | LogID (PK) INT AUTO_INCREMENT, ItemID (FK) INT, CompanyID (FK) INT, ErrorMessage VARCHAR(255) NOT NULL, LogTime DATETIME NOT NULL DEFAULT NOW(), FOREIGN KEY (ItemID) REFERENCES Item(ItemID), FOREIGN KEY (CompanyID) REFERENCES Company(CompanyID) |
| **ItemLogErrors** | LogID (PK) INT AUTO_INCREMENT, ItemID (FK) INT, AisleID (FK) INT, ErrorMessage VARCHAR(255) NOT NULL, SuggestedAisle INT, LogTime DATETIME NOT NULL DEFAULT NOW(), FOREIGN KEY (ItemID) REFERENCES Item(ItemID), FOREIGN KEY (AisleID) REFERENCES Aisle(AisleID) |


## Normalizzazione dello Schema Logico

### 1. Prima Forma Normale (1NF)

La **1NF** richiede che ogni tabella abbia un attributo atomico e che non ci siano gruppi di ripetizione o set di valori multipli.

- Le tabelle esistenti sono già essere nella 1NF, poiché non vi sono colonne che contengono set di valori multipli o gruppi di ripetizione. Gli attributi sono singoli e ogni colonna contiene valori atomici.

### 2. Seconda Forma Normale (2NF)

La **2NF** richiede che tutte le tabelle siano nella 1NF e che ogni attributo non chiave dipenda completamente dalla chiave primaria (nessuna dipendenza parziale).

- Le tabelle **ItemLog** e **ItemLogErrors** contengono dipendenze parziali, dove l'attributo **ErrorMessage** non dipende dalla chiave primaria, ma solo da uno dei suoi componenti.

- Creo una tabella separato per gli **ErrorMessages** e lo collego tramite chiavi esterne **ErrorID**.

#### Risultato della 2NF:
- **ItemLog**:
  - Chiave primaria: **LogID**
  - Colonne: **ItemID**, **CompanyID**, **LogTime**
- **ItemLogErrors**:
  - Chiave primaria: **LogID**
  - Colonne: **ItemID**, **AisleID**, **LogTime**, **SuggestedAisle**
  
- **ErrorMessages** (nuova tabella):
  - Chiave primaria: **ErrorID**
  - Colonne: **ErrorMessage**

### 3. Terza Forma Normale (3NF)

La **3NF** richiede che tutte le tabelle siano nella 2NF e che non ci siano dipendenze transitive, ovvero che gli attributi non dipendano da altri attributi non chiave.

#### Problemi rilevati:
- In **ItemLogErrors**, il campo **SuggestedAisle** dipende da **AisleID**. Tuttavia, esso é una tabella temporale per cui non crea problemi.


### Schema Logico Normalizzato

Ecco la versione normalizzata dello schema logico:

#### 1. **Aisle (Corsia)**
| Attributo  | Tipo        | Note               |
|------------|-------------|--------------------|
| AisleID    | INT (PK)    | Identificativo univoco della corsia |
| Name       | VARCHAR     | Nome della corsia  |

#### 2. **Item (Oggetto)**
| Attributo  | Tipo        | Note               |
|------------|-------------|--------------------|
| ItemID     | INT (PK)    | Identificativo univoco dell'oggetto |
| Name       | VARCHAR     | Nome dell'oggetto  |
| Category   | VARCHAR     | Categoria dell'oggetto |
| StorageType| VARCHAR     | Tipo di conservazione dell'oggetto |
| ExpirationDate | DATE    | Data di scadenza dell'oggetto |

#### 3. **Company (Azienda)**
| Attributo  | Tipo        | Note               |
|------------|-------------|--------------------|
| CompanyID  | INT (PK)    | Identificativo univoco dell'azienda |
| Name       | VARCHAR     | Nome dell'azienda |
| Email      | VARCHAR     | Email dell'azienda |
| Location   | VARCHAR     | Posizione dell'azienda |

#### 4. **ItemLog (Registro Oggetti)**
| Attributo  | Tipo        | Note               |
|------------|-------------|--------------------|
| LogID      | INT (PK)    | Identificativo univoco del log |
| ItemID     | INT (FK)    | Riferimento all'oggetto (chiave esterna) |
| CompanyID  | INT (FK)    | Riferimento all'azienda (chiave esterna) |
| LogTime    | DATETIME    | Data e ora del log |
| ErrorID    | INT (FK)    | Riferimento all'errore (chiave esterna) |

#### 5. **ItemLogErrors (Errori nel Registro Oggetti)**
| Attributo  | Tipo        | Note               |
|------------|-------------|--------------------|
| LogID      | INT (PK)    | Identificativo univoco del log |
| ItemID     | INT (FK)    | Riferimento all'oggetto (chiave esterna) |
| SuggestedAisle    | INT    | Riferimento alla corsia migliore |
| LogTime    | DATETIME    | Data e ora del log |
| ErrorID    | INT (FK)    | Riferimento all'errore (chiave esterna) |

#### 6. **ErrorMessages (Messaggi di Errore)**
| Attributo  | Tipo        | Note               |
|------------|-------------|--------------------|
| ErrorID    | INT (PK)    | Identificativo univoco dell'errore |
| ErrorMessage | VARCHAR   | Descrizione dell'errore |


---

![Getting Started](./Aisle_Manegment.png)